# Section 6: Estimator Primitive & VQE/QAOA - Practice Questions

**Exam Weight**: 12% (~8 questions) | **Difficulty**: High | **Must Master**: ✅✅✅✅

---

## 🎯 Key Traps to Watch For:

| Trap | Wrong Assumption | Correct Understanding |
|------|------------------|----------------------|
| Estimator circuits | Has measurements | NO measurements! Estimator measures observables |
| Observable format | String `"ZZ"` | `SparsePauliOp("ZZ")` or list of them |
| Pauli ordering | Left-to-right | RIGHT-TO-LEFT: `"ZI"` = Z⊗I = Z on q1, I on q0 |
| Result values | `.evs` singular | `.evs` is array - use `evs[0]` or iterate |
| VQE parameters | Fixed values | `ParameterVector` bound during optimization |
| QAOA mixer | Custom always | Standard X mixer is default |

> 📖 See section_6_estimator/README.md for full concepts

---

## 📚 Topics Covered (from Section Notebooks):

### Estimator Primitive (`estimator_primitive.ipynb`)

#### StatevectorEstimator (Local)
- **Creation**: `from qiskit.primitives import StatevectorEstimator`
- **Running**: `estimator.run([(circuit, observable)])`
- **NO measurements**: Circuit must NOT have measurements!

#### EstimatorV2 (Runtime)
- **Creation**: `from qiskit_ibm_runtime import EstimatorV2`
- **PUB format**: `(circuit, observable, parameter_values, precision)`

#### SparsePauliOp
- **Creation**: `SparsePauliOp('ZZ')`, `SparsePauliOp(['ZZ', 'XX'], [0.5, 0.5])`
- **Pauli ordering**: RIGHT-TO-LEFT (rightmost = q0)
- **Operations**: `.compose()`, `@` operator for tensor product

#### Result Extraction
- **Expectation values**: `result[0].data.evs`
- **Standard deviations**: `result[0].data.stds`
- **Array access**: `evs[0]` for first observable

### VQE Pattern (`vqe_pattern.ipynb`)

#### Variational Quantum Eigensolver
- **Ansatz**: `TwoLocal`, `EfficientSU2`, `RealAmplitudes`
- **Optimizer**: `scipy.optimize.minimize`
- **Cost function**: Estimator expectation value

#### QAOA Pattern
- **Problem encoding**: MaxCut graph to Hamiltonian
- **Cost layer**: Problem Hamiltonian
- **Mixer layer**: X gates (standard mixer)

#### Error Mitigation
- **ZNE**: Zero-noise extrapolation
- **M3**: Matrix-free measurement mitigation

In [ ]:
# Setup - Run this first!
from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp, Pauli
from qiskit.circuit import Parameter, ParameterVector
from qiskit.circuit.library import RealAmplitudes
import numpy as np
%matplotlib inline
print("✅ Setup complete!")

---
## Part 1: SparsePauliOp Basics

| Constructor | Example |
|-------------|--------|
| `SparsePauliOp('ZZ')` | Single Pauli term |
| `SparsePauliOp(['ZZ', 'XX'], [0.5, 0.5])` | Weighted sum |
| `SparsePauliOp.from_list([('ZZ', 0.5)])` | From list of tuples |

### Q1: Create SparsePauliOp

In [ ]:
# Your solution: Create H = 0.5*ZZ + 0.3*XX + 0.2*YY

In [ ]:
# Solution Q1
# Method 1: From lists
# H1 = SparsePauliOp(['ZZ', 'XX', 'YY'], coeffs=[0.5, 0.3, 0.2])
H1 = SparsePauliOp(['ZZ', 'XX', 'YY'], coeffs = [0.5, 0.3, 0.2])
print("Method 1:")
print(H1)

# Method 2: from_list
H2 = SparsePauliOp.from_list([('ZZ', 0.5), ('XX', 0.3), ('YY', 0.2)])
print("\nMethod 2:")
print(H2)

print(f"\nNumber of qubits: {H1.num_qubits}")
print(f"Number of terms: {len(H1)}")

### Q2: SparsePauliOp arithmetic

In [ ]:
# Your solution: Add and multiply SparsePauliOps

In [ ]:
# Solution Q2
op1 = SparsePauliOp('ZZ')
op2 = SparsePauliOp('XX')

# Addition
sum_op = op1 + op2
print(f"ZZ + XX = {sum_op}")

# Scalar multiplication
scaled = 0.5 * op1
print(f"\n0.5 * ZZ = {scaled}")

# Combination
H = 0.5 * SparsePauliOp('ZZ') + 0.3 * SparsePauliOp('XX')
print(f"\nH = {H}")

---
## Part 2: StatevectorEstimator Basics

⚠️ **EXAM CRITICAL**: Estimator computes expectation values ⟨ψ|O|ψ⟩

| Key Difference | Sampler | Estimator |
|----------------|---------|----------|
| Output | Counts/probs | Expectation value |
| Circuit | Needs measurement | NO measurement |
| PUB format | `(circuit,)` | `(circuit, observable)` |

### Q3: Basic Estimator usage

In [ ]:
# Your solution: Compute ⟨Z⟩ for |0⟩ state

In [ ]:
# Solution Q3
# Create circuit (NO measurement for Estimator!)
qc = QuantumCircuit(1)
# |0⟩ state - no gates needed

# Observable to measure
observable = SparsePauliOp('Z')

# Create estimator and run
estimator = StatevectorEstimator()

# PUB format: (circuit, observable)
job = estimator.run([(qc, observable)])
result = job.result()

# Extract expectation value
exp_val = result[0].data.evs
print(f"⟨0|Z|0⟩ = {exp_val}")
print("Expected: 1.0 (|0⟩ is +1 eigenstate of Z)")

### Q4: Expectation value for superposition

In [ ]:
# Your solution: Compute ⟨Z⟩ and ⟨X⟩ for |+⟩ state

In [ ]:
# Solution Q4
# Create |+⟩ = H|0⟩
qc = QuantumCircuit(1)
qc.h(0)

estimator = StatevectorEstimator()

# Measure Z
result_z = estimator.run([(qc, SparsePauliOp('Z'))]).result()
print(f"⟨+|Z|+⟩ = {result_z[0].data.evs}")
print("Expected: 0.0 (|+⟩ is NOT eigenstate of Z)")

# Measure X
result_x = estimator.run([(qc, SparsePauliOp('X'))]).result()
print(f"\n⟨+|X|+⟩ = {result_x[0].data.evs}")
print("Expected: 1.0 (|+⟩ IS +1 eigenstate of X)")

---
## Part 3: Multi-Qubit Observables

### Q5: Two-qubit observables

In [ ]:
# Your solution: Compute ⟨ZZ⟩ for Bell state

In [ ]:
# Solution Q5
# Create Bell state |Φ+⟩ = (|00⟩ + |11⟩)/√2
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)

estimator = StatevectorEstimator()

# ZZ correlation
result_zz = estimator.run([(qc, SparsePauliOp('ZZ'))]).result()
print(f"⟨Φ+|ZZ|Φ+⟩ = {result_zz[0].data.evs}")
print("Expected: 1.0 (perfect ZZ correlation)")

# XX correlation
result_xx = estimator.run([(qc, SparsePauliOp('XX'))]).result()
print(f"\n⟨Φ+|XX|Φ+⟩ = {result_xx[0].data.evs}")
print("Expected: 1.0 (perfect XX correlation)")

### Q6: Hamiltonian with multiple terms

In [ ]:
# Your solution: Measure Heisenberg Hamiltonian H = J(XX + YY + ZZ)

In [ ]:
# Solution Q6
# Heisenberg Hamiltonian
J = 1.0
H = J * (SparsePauliOp('XX') + SparsePauliOp('YY') + SparsePauliOp('ZZ'))
print(f"Hamiltonian: {H}")

# Bell state
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)

estimator = StatevectorEstimator()
result = estimator.run([(qc, H)]).result()

energy = result[0].data.evs
print(f"\n⟨H⟩ = {energy}")
print("Expected: 1.0 (Bell state is eigenstate)")

---
## Part 4: Parameterized Circuits

### Q7: Estimate with parameters

In [ ]:
# Your solution: Run parameterized circuit with Estimator

In [ ]:
# Solution Q7
theta = Parameter('θ')

qc = QuantumCircuit(1)
qc.ry(theta, 0)

observable = SparsePauliOp('Z')
estimator = StatevectorEstimator()

# Test different angles
angles = [0, np.pi/4, np.pi/2, np.pi]
print("⟨Z⟩ vs RY angle:")

for angle in angles:
    # PUB with params: (circuit, observable, params)
    result = estimator.run([(qc, observable, [angle])]).result()
    exp_val = result[0].data.evs
    print(f"  θ={angle:.2f}: ⟨Z⟩ = {exp_val:.4f}")

print("\nNote: RY(π/2)|0⟩ = |+⟩ → ⟨Z⟩ = 0")

---
## Part 5: VQE Pattern

VQE workflow:
1. Create parameterized ansatz
2. Define Hamiltonian (SparsePauliOp)
3. Use Estimator to compute ⟨H⟩
4. Optimize parameters to minimize energy

### Q8: VQE-style energy landscape

In [ ]:
# Your solution: Scan energy vs parameter for simple ansatz

In [ ]:
# Solution Q8
# Simple ansatz: RY rotation
theta = Parameter('θ')
ansatz = QuantumCircuit(1)
ansatz.ry(theta, 0)

# "Hamiltonian": just Z operator
H = SparsePauliOp('Z')

estimator = StatevectorEstimator()

# Energy landscape
angles = np.linspace(0, 2*np.pi, 20)
energies = []

for angle in angles:
    result = estimator.run([(ansatz, H, [angle])]).result()
    energies.append(result[0].data.evs)

# Find minimum
min_idx = np.argmin(energies)
print(f"Energy landscape scanned")
print(f"Minimum energy: {energies[min_idx]:.4f} at θ={angles[min_idx]:.2f}")
print(f"Expected: -1.0 at θ=π (|1⟩ state)")

### Q9: Multi-parameter VQE ansatz

In [ ]:
# Your solution: Create 2-qubit VQE ansatz and evaluate energy

In [ ]:
# Solution Q9
from qiskit.circuit import ParameterVector

# VQE ansatz with entanglement
params = ParameterVector('θ', 4)
ansatz = QuantumCircuit(2)
ansatz.ry(params[0], 0)
ansatz.ry(params[1], 1)
ansatz.cx(0, 1)
ansatz.ry(params[2], 0)
ansatz.ry(params[3], 1)

print("VQE Ansatz:")
print(ansatz.draw())

# Hamiltonian
H = SparsePauliOp.from_list([('ZZ', 1.0), ('XX', 0.5)])

# Evaluate at random parameters
estimator = StatevectorEstimator()
random_params = np.random.uniform(0, 2*np.pi, 4)

result = estimator.run([(ansatz, H, random_params)]).result()
print(f"\nEnergy at random params: {result[0].data.evs:.4f}")

---
## Part 6: Multiple Observables

### Q10: Run multiple observables in one job

In [ ]:
# Your solution: Measure X, Y, Z for same state in one job

In [ ]:
# Solution Q10
# Create |+⟩ state
qc = QuantumCircuit(1)
qc.h(0)

# Multiple observables
obs_x = SparsePauliOp('X')
obs_y = SparsePauliOp('Y')
obs_z = SparsePauliOp('Z')

estimator = StatevectorEstimator()

# Multiple PUBs in one job
result = estimator.run([
    (qc, obs_x),
    (qc, obs_y),
    (qc, obs_z)
]).result()

print("Expectation values for |+⟩:")
print(f"  ⟨X⟩ = {result[0].data.evs:.4f} (expected: 1.0)")
print(f"  ⟨Y⟩ = {result[1].data.evs:.4f} (expected: 0.0)")
print(f"  ⟨Z⟩ = {result[2].data.evs:.4f} (expected: 0.0)")

---
## ✅ Section 6 Checklist

**SparsePauliOp**:
- [ ] `SparsePauliOp('ZZ')` - single term
- [ ] `SparsePauliOp(['ZZ', 'XX'], coeffs=[a, b])` - weighted sum
- [ ] `SparsePauliOp.from_list([('ZZ', coef), ...])` - from list
- [ ] Arithmetic: `+`, `*` (scalar)

**StatevectorEstimator**:
- [ ] `StatevectorEstimator()` - create estimator
- [ ] NO measurement in circuit!
- [ ] PUB: `(circuit, observable)` or `(circuit, observable, params)`

**Result Extraction**:
- [ ] `result[pub_idx].data.evs` - expectation value
- [ ] `result[pub_idx].data.stds` - standard deviation

**VQE Pattern**:
- [ ] Parameterized ansatz (ParameterVector)
- [ ] Hamiltonian as SparsePauliOp
- [ ] Estimator computes ⟨ψ(θ)|H|ψ(θ)⟩
- [ ] Optimize θ to minimize energy

**Key Differences from Sampler**:
- [ ] Sampler: Needs measurement, returns counts
- [ ] Estimator: No measurement, returns ⟨O⟩